# pyvista sandbox
thomas.moore@csiro.au - 15 July 2022

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# references

https://vtk.org/ <br>
https://docs.pyvista.org/ <br>
https://github.com/pyvista <br>
https://github.com/pyvista/pyvista-xarray <br>

### install notes

#### ``` conda install -c conda-forge pyvista-xarray ``` 
#### ``` conda install -c conda-forge ipyvtklink ```

# import packages

In [2]:
import numpy as np
import xarray as xr
import pvxarray

# functions

In [3]:
def calc_EKE(u, v):
    '''
    Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}
    u,v are x and y currents as an xarray data array
    '''
    u_mean = u.mean('time')
    v_mean = v.mean('time')
    MKE = 0.5*(u_mean**2 + v_mean**2).rename('MKE') # currents
    EKE = ( 0.5 * ((u-u_mean)**2 + (v-v_mean)**2) ).rename('EKE') # eddies
    return EKE, MKE

# load data and do some visualisation

In [4]:
data_file = '/Users/moo270/cloudstor/ETBF_CAFE_data/CAFE60_legacy_vars/CAFE60_ETBF_legacy_vars.nc'

In [5]:
ds_2D = xr.open_dataset(data_file)

In [6]:
ds_2D

<xarray.Dataset>
Dimensions:    (time: 360, longitude: 90, latitude: 111)
Coordinates:
  * time       (time) object 1990-01-16 12:00:00 ... 2019-12-16 12:00:00
  * longitude  (longitude) float64 140.5 141.5 142.5 143.5 ... 227.5 228.5 229.5
  * latitude   (latitude) float64 -69.63 -69.18 -68.71 ... -0.5005 -0.1662
Data variables:
    SST        (time, latitude, longitude) float32 ...
    SSS        (time, latitude, longitude) float32 ...
    mld        (time, latitude, longitude) float32 ...
    temp50     (time, latitude, longitude) float32 ...
    temp100    (time, latitude, longitude) float32 ...
    temp200    (time, latitude, longitude) float32 ...
    temp500    (time, latitude, longitude) float32 ...

In [7]:
snapshot_2D = ds_2D.isel(time=180)

In [8]:
mesh = snapshot_2D.SST.pyvista.mesh(x="longitude", y="latitude")
mesh.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# local dask cluster

In [9]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default

In [10]:
zarr_file = '/Users/moo270/data/accessS2.RA.ocean.masked.AUSWCPregion.TUV_025deg.zarr'

In [11]:
ds_3D = xr.open_zarr(zarr_file)
ds_3D

<xarray.Dataset>
Dimensions:        (lat: 361, lon: 721, time: 492, depth: 75)
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * lat            (lat) float64 -70.0 -69.75 -69.5 -69.25 ... 19.5 19.75 20.0
  * lon            (lon) float64 90.0 90.25 90.5 90.75 ... 269.5 269.8 270.0
  * time           (time) datetime64[ns] 1981-01-15 1981-02-15 ... 2021-12-15
Data variables: (12/20)
    areat          (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    areau          (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    areav          (lat, lon) float32 dask.array<chunksize=(361, 721), meta=np.ndarray>
    d20            (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    hc300          (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    maskT          (lat, lon) int64 dask.array<chunksize=(361, 721), meta=np.ndarray>
    ...             ...
    temp100        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp200        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp50         (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    temp500        (time, lat, lon) float32 dask.array<chunksize=(12, 361, 721), meta=np.ndarray>
    u              (time, depth, lat, lon) float32 dask.array<chunksize=(1, 75, 361, 721), meta=np.ndarray>
    v              (time, depth, lat, lon) float32 dask.array<chunksize=(1, 75, 361, 721), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [12]:
u = ds_3D.u
v = ds_3D.v

# Calculate EKE / MKE

In [13]:
[EKE, MKE] = calc_EKE(u,v)

### size of objects

In [14]:
EKE.nbytes/1e9

38.4174756

In [15]:
MKE.nbytes/1e9

0.0780843

In [18]:
EKE.sel(time='2021-11-15').nbytes/1e9

0.0780843

In [16]:
%%time
EKE_snapshot = EKE.sel(time='2021-11-15').compute()

CPU times: user 27.3 s, sys: 5.17 s, total: 32.5 s
Wall time: 1min 47s


In [19]:
%%time
MKE = MKE.compute()

CPU times: user 20.4 s, sys: 3.42 s, total: 23.8 s
Wall time: 1min 29s


### trigger garbage collection - clear unmanaged memory

In [20]:
import gc
client.run(gc.collect)  # collect garbage on all workers

{'tcp://127.0.0.1:64889': 818,
 'tcp://127.0.0.1:64892': 671,
 'tcp://127.0.0.1:64895': 1138,
 'tcp://127.0.0.1:64898': 734}

# invert depth axis

In [21]:
EKE_snapshot['depth'] = EKE_snapshot['depth']*-1
MKE['depth'] = MKE['depth']*-1

# what is range of values

In [22]:
EKE_snapshot.to_dataframe().describe()

,EKE
count,1.482161e+07
mean,7.563711e-03
std,1.958111e-02
min,2.109580e-12
25%,3.537016e-04
50%,1.519920e-03
75%,5.961645e-03
max,1.435385e+00


In [23]:
cmap_range = [5.961645e-03,0.2]

In [24]:
mesh = EKE_snapshot.pyvista.mesh(x='lon', y='lat',z='depth')
mesh.plot(cmap='inferno',rng=cmap_range)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## can we render the volume?

```volume: bool, optional.  
Use the Plotter.add_volume() method for volume rendering.```

In [25]:
mesh.plot(volume=True)

TypeError: Type <class 'pyvista.core.grid.RectilinearGrid'> not supported for volume rendering at this time. Use `pyvista.UniformGrid`.

# https://docs.pyvista.org/api/core/_autosummary/pyvista.UniformGrid.html?highlight=uniformgrid#pyvista.UniformGrid